In [1]:
import requests

In [2]:
def get_access_token(client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    payload = {
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, headers=headers, data=payload, auth=(client_id, client_secret))
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception("Failed to retrieve access token")


In [3]:
# Replace 'your_client_id' and 'your_client_secret' with your actual Spotify Client ID and Secret
client_id = '<client id>'
client_secret =  '<client secret>'
access_token = get_access_token(client_id, client_secret)

Exception: Failed to retrieve access token

In [ ]:
def get_artist_data(access_token, artist_id):
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to retrieve artist data")


successfull call to access metadata for an artist on spotify

In [6]:
artist_id = '1vCWHaC5f2uS3yhpwWbIA6'  # This is the artist ID for Avicii
artist_data = get_artist_data(access_token, artist_id)
print(artist_data)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1vCWHaC5f2uS3yhpwWbIA6'}, 'followers': {'href': None, 'total': 22669560}, 'genres': ['dance pop', 'edm', 'pop', 'pop dance'], 'href': 'https://api.spotify.com/v1/artists/1vCWHaC5f2uS3yhpwWbIA6', 'id': '1vCWHaC5f2uS3yhpwWbIA6', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebae07171f989fb39736674113', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174ae07171f989fb39736674113', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178ae07171f989fb39736674113', 'width': 160}], 'name': 'Avicii', 'popularity': 81, 'type': 'artist', 'uri': 'spotify:artist:1vCWHaC5f2uS3yhpwWbIA6'}
